In [1]:
import pikepdf
import sys
import os
import re
from xfaTools import XfaObj
import xml.etree.ElementTree as ET
from lxml import etree

In [2]:
import pypdf

In [2]:
fileName = r"SF424B-V1.1 (1).pdf"
#fileName = "test_424.pdf"
pdfData = pikepdf.Pdf.open(fileName)

In [3]:
fileName = r"SF424B-V1.1 (1).pdf"
pdfData = pypdf.PdfReader(fileName)

In [4]:
xfaDict = XfaObj(pdfData)
xfaDict

{'preamble': '<?xml version="1.0" encoding="UTF-8"?>\n<xdp:xdp xmlns:xdp="http://ns.adobe.com/xdp/" timeStamp="2022-03-04T18:02:38Z" uuid="4fbd55ae-4a39-47f3-b167-0f0f223925c7">\n',
 'config': '<config xmlns="http://www.xfa.org/schema/xci/3.0/"\n><agent name="designer"\n><!--  [0..n]  --><destination\n>pdf</destination\n><pdf\n><!--  [0..n]  --><fontInfo\n/></pdf\n></agent\n><present\n><!--  [0..n]  --><pdf\n><!--  [0..n]  --><version\n>1.7</version\n><adobeExtensionLevel\n>8</adobeExtensionLevel\n><renderPolicy\n>client</renderPolicy\n><creator\n>Designer 6.4</creator\n><producer\n>Designer 6.4</producer\n><scriptModel\n>XFA</scriptModel\n><interactive\n>1</interactive\n><tagged\n>1</tagged\n><fontInfo\n><embed\n>0</embed\n></fontInfo\n><compression\n><level\n>6</level\n><compressLogicalStructure\n>1</compressLogicalStructure\n><compressObjectStream\n>1</compressObjectStream\n></compression\n><linearized\n>1</linearized\n><silentPrint\n><addSilentPrint\n>0</addSilentPrint\n><printerNa

In [6]:
pdfData.xfa

{'preamble': b'<?xml version="1.0" encoding="UTF-8"?>\n<xdp:xdp xmlns:xdp="http://ns.adobe.com/xdp/" timeStamp="2022-03-04T18:02:38Z" uuid="4fbd55ae-4a39-47f3-b167-0f0f223925c7">\n',
 'config': b'<config xmlns="http://www.xfa.org/schema/xci/3.0/"\n><agent name="designer"\n><!--  [0..n]  --><destination\n>pdf</destination\n><pdf\n><!--  [0..n]  --><fontInfo\n/></pdf\n></agent\n><present\n><!--  [0..n]  --><pdf\n><!--  [0..n]  --><version\n>1.7</version\n><adobeExtensionLevel\n>8</adobeExtensionLevel\n><renderPolicy\n>client</renderPolicy\n><creator\n>Designer 6.4</creator\n><producer\n>Designer 6.4</producer\n><scriptModel\n>XFA</scriptModel\n><interactive\n>1</interactive\n><tagged\n>1</tagged\n><fontInfo\n><embed\n>0</embed\n></fontInfo\n><compression\n><level\n>6</level\n><compressLogicalStructure\n>1</compressLogicalStructure\n><compressObjectStream\n>1</compressObjectStream\n></compression\n><linearized\n>1</linearized\n><silentPrint\n><addSilentPrint\n>0</addSilentPrint\n><printer

In [5]:
root = etree.fromstring(xfaDict['template'])
ns = {'xfa':"http://www.xfa.org/schema/xfa-template/3.3/"}



In [6]:
# remove signature form field
for field in root.findall(".//xfa:subform[@name='SF424B_P2']/xfa:field", ns):
    print(field.attrib)
    if field.get('name') == 'Signature':
        field.getparent().remove(field)

print("\n")

for field in root.findall(".//xfa:subform[@name='SF424B_P2']/xfa:field", ns):
    print(field.attrib)

{'h': '6mm', 'name': 'Mandatory', 'presence': 'invisible', 'w': '20.9126mm', 'x': '44.9698mm', 'y': '4.7401mm'}
{'h': '11.64mm', 'name': 'Signature', 'w': '102.9324mm', 'x': '13.4874mm', 'y': '193.66mm'}
{'h': '4.7978mm', 'name': 'AuthorizedSignatureTitle', 'w': '79.9148mm', 'x': '119.9134mm', 'y': '193.66mm'}
{'h': '4.7978mm', 'name': 'ORGANIZATION', 'w': '103.19mm', 'x': '13.7414mm', 'y': '213.45mm'}
{'h': '4.7978mm', 'name': 'SubmittedDate', 'w': '79.9084mm', 'x': '119.634mm', 'y': '213.45mm'}
{'h': '9mm', 'name': 'FormVersionIdentifier', 'presence': 'hidden', 'w': '62mm', 'x': '26.99mm', 'y': '247.36mm'}
{'h': '0.0001mm', 'name': 'LastField', 'w': '0.0001mm', 'x': '143.3999mm', 'y': '217.69mm'}


{'h': '6mm', 'name': 'Mandatory', 'presence': 'invisible', 'w': '20.9126mm', 'x': '44.9698mm', 'y': '4.7401mm'}
{'h': '4.7978mm', 'name': 'AuthorizedSignatureTitle', 'w': '79.9148mm', 'x': '119.9134mm', 'y': '193.66mm'}
{'h': '4.7978mm', 'name': 'ORGANIZATION', 'w': '103.19mm', 'x': '13.74

In [7]:
# create the signature field
field = etree.Element("{http://www.xfa.org/schema/xfa-template/3.3/}field", attrib = {'h': '11.64mm', 'name': 'Signature', 'w': '102.9324mm', 'x': '13.4874mm', 'y': '193.66mm'})
ui = etree.SubElement(field, "{http://www.xfa.org/schema/xfa-template/3.3/}ui")
sig = etree.SubElement(ui, "{http://www.xfa.org/schema/xfa-template/3.3/}signature", attrib = {'type':'PDF1.6'})
man = etree.SubElement(sig, "{http://www.xfa.org/schema/xfa-template/3.3/}manifest", attrib = {'action':'all'})
fil = etree.SubElement(sig, "{http://www.xfa.org/schema/xfa-template/3.3/}filter")
loc = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}lockDocument")#, attrib = {'type': 'required'})
loc.text = "1"
#mdp = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}mdp", attrib = {'permissions': '1', 'signatureType': 'author'})
print(etree.tostring(field))

b'<ns0:field xmlns:ns0="http://www.xfa.org/schema/xfa-template/3.3/" h="11.64mm" name="Signature" w="102.9324mm" x="13.4874mm" y="193.66mm"><ns0:ui><ns0:signature type="PDF1.6"><ns0:manifest action="all"/><ns0:filter><ns0:lockDocument>1</ns0:lockDocument></ns0:filter></ns0:signature></ns0:ui></ns0:field>'


In [8]:
# now add the signature field
root.find(".//xfa:subform[@name='SF424B_P2']", ns).append(field)

for field in root.findall(".//xfa:subform[@name='SF424B_P2']/xfa:field", ns):
    print(field.attrib)

{'h': '6mm', 'name': 'Mandatory', 'presence': 'invisible', 'w': '20.9126mm', 'x': '44.9698mm', 'y': '4.7401mm'}
{'h': '4.7978mm', 'name': 'AuthorizedSignatureTitle', 'w': '79.9148mm', 'x': '119.9134mm', 'y': '193.66mm'}
{'h': '4.7978mm', 'name': 'ORGANIZATION', 'w': '103.19mm', 'x': '13.7414mm', 'y': '213.45mm'}
{'h': '4.7978mm', 'name': 'SubmittedDate', 'w': '79.9084mm', 'x': '119.634mm', 'y': '213.45mm'}
{'h': '9mm', 'name': 'FormVersionIdentifier', 'presence': 'hidden', 'w': '62mm', 'x': '26.99mm', 'y': '247.36mm'}
{'h': '0.0001mm', 'name': 'LastField', 'w': '0.0001mm', 'x': '143.3999mm', 'y': '217.69mm'}
{'h': '11.64mm', 'name': 'Signature', 'w': '102.9324mm', 'x': '13.4874mm', 'y': '193.66mm'}


In [9]:
# update pdf data
xfaDict['template'] = etree.tostring(root)

In [16]:
pdfData.xfa = xfaDict

AttributeError: can't set attribute

In [12]:
pdfDataW = pypdf.PdfWriter()
pdfDataW.clone_document_from_reader(pdfData)
#pdfDataW.xfa = xfaDict
with open("document-output-clone_v2.pdf", "wb") as f:
    pdfDataW.write(f)



In [10]:
pdfData.close()

In [11]:
pdfData = pikepdf.Pdf.open("SF424D-V1.1.pdf")
xfaDict = XfaObj(pdfData)
root = etree.fromstring(xfaDict['template'])
ns = {'xfa':"http://www.xfa.org/schema/xfa-template/3.3/"}

In [7]:
# remove signature form field
for field in root.findall(".//xfa:subform[@name='SF424D_P2']/xfa:field", ns):
    print(field.attrib)
    if field.get('name') == 'Signature':
        field.getparent().remove(field)

print("\n")

for field in root.findall(".//xfa:subform[@name='SF424D_P2']/xfa:field", ns):
    print(field.attrib)

{'h': '4.7978mm', 'name': 'Title', 'w': '81.396mm', 'x': '119.7864mm', 'y': '178.86mm'}
{'h': '4.7978mm', 'name': 'Organization', 'w': '103.4902mm', 'x': '12.9198mm', 'y': '197.53mm'}
{'h': '6mm', 'name': 'CloseForm', 'presence': 'invisible', 'w': '26.4601mm', 'x': '22.76mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'GotoPreviousPage', 'presence': 'invisible', 'w': '22.49mm', 'x': '89.7002mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'GotoNextPage', 'presence': 'invisible', 'w': '22.7602mm', 'x': '114.5702mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'PrintButton', 'presence': 'invisible', 'w': '24.07mm', 'x': '153.981mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'AboutButton', 'presence': 'invisible', 'w': '24.07mm', 'x': '191.0662mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'Mandatory', 'presence': 'invisible', 'w': '20.9126mm', 'x': '56.15mm', 'y': '5.5301mm'}
{'h': '11.6399mm', 'name': 'Signature', 'w': '103.2302mm', 'x': '12.9198mm', 'y': '178.86mm'}
{'h': '4.7978mm', 'name': 'SubmittedDate', '

In [9]:
# create the signature field
field = etree.Element("{http://www.xfa.org/schema/xfa-template/3.3/}field", attrib = {'h': '11.6399mm', 'name': 'Signature', 'w': '103.2302mm', 'x': '12.9198mm', 'y': '178.86mm'}
)
ui = etree.SubElement(field, "{http://www.xfa.org/schema/xfa-template/3.3/}ui")
sig = etree.SubElement(ui, "{http://www.xfa.org/schema/xfa-template/3.3/}signature", attrib = {'type':'PDF1.6'})
fil = etree.SubElement(sig, "{http://www.xfa.org/schema/xfa-template/3.3/}filter")
loc = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}lockDocument")#, attrib = {'type': 'required'})
loc.text = "1"
#mdp = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}mdp", attrib = {'permissions': '1', 'signatureType': 'author'})
print(etree.tostring(field))

b'<ns0:field xmlns:ns0="http://www.xfa.org/schema/xfa-template/3.3/" h="11.6399mm" name="Signature" w="103.2302mm" x="12.9198mm" y="178.86mm"><ns0:ui><ns0:signature type="PDF1.6"><ns0:filter><ns0:lockDocument>1</ns0:lockDocument></ns0:filter></ns0:signature></ns0:ui></ns0:field>'


In [10]:
# now add the signature field
root.find(".//xfa:subform[@name='SF424D_P2']", ns).append(field)

for field in root.findall(".//xfa:subform[@name='SF424D_P2']/xfa:field", ns):
    print(field.attrib)

{'h': '4.7978mm', 'name': 'Title', 'w': '81.396mm', 'x': '119.7864mm', 'y': '178.86mm'}
{'h': '4.7978mm', 'name': 'Organization', 'w': '103.4902mm', 'x': '12.9198mm', 'y': '197.53mm'}
{'h': '6mm', 'name': 'CloseForm', 'presence': 'invisible', 'w': '26.4601mm', 'x': '22.76mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'GotoPreviousPage', 'presence': 'invisible', 'w': '22.49mm', 'x': '89.7002mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'GotoNextPage', 'presence': 'invisible', 'w': '22.7602mm', 'x': '114.5702mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'PrintButton', 'presence': 'invisible', 'w': '24.07mm', 'x': '153.981mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'AboutButton', 'presence': 'invisible', 'w': '24.07mm', 'x': '191.0662mm', 'y': '5.5302mm'}
{'h': '6mm', 'name': 'Mandatory', 'presence': 'invisible', 'w': '20.9126mm', 'x': '56.15mm', 'y': '5.5301mm'}
{'h': '4.7978mm', 'name': 'SubmittedDate', 'w': '81.4578mm', 'x': '119.7864mm', 'y': '197.53mm'}
{'h': '9mm', 'name': 'FormVersionIdentifi

In [12]:
# update pdf data
xfaDict['template'] = etree.tostring(root)
pdfData.save("SF424D_signable.pdf")

In [ ]:
pdfData.close()

In [15]:
pdfData = pikepdf.Pdf.open("SFLLL_2_0-V2.0.pdf")
xfaDict = XfaObj(pdfData)
root = etree.fromstring(xfaDict['template'])
ns = {'xfa':"http://www.xfa.org/schema/xfa-template/3.3/"}

In [18]:
for field in root.findall(".//xfa:subform", ns):
    print(field.attrib)

{'layout': 'tb', 'name': 'GrantApplicationWrapper'}
{'layout': 'tb', 'name': 'SFLLL_2_0_Main', 'w': '8.5in'}
{'h': '11.000in', 'name': 'SFLLL_2_0_P1', 'w': '8.500in'}
{'minH': '28.13mm', 'w': '45.782mm', 'x': '24.655mm', 'y': '30.601mm'}
{'minH': '17.51mm', 'w': '48.71mm', 'x': '82.92mm', 'y': '30.601mm'}
{'layout': 'tb', 'w': '1.520674in', 'x': '151.679mm', 'y': '43.741mm'}
{'name': 'MaterialChangeSupplement'}
{'h': '32.812mm', 'w': '172.908mm', 'x': '24.724mm', 'y': '60.712mm'}
{'x': '4.307mm', 'y': '3.997mm'}
{'layout': 'tb', 'x': '40.561mm', 'y': '3.997mm'}
{'name': 'Tier'}
{'minH': '22.726mm', 'w': '166.979mm', 'x': '0.188mm', 'y': '10.086mm'}
{'w': '166.7274mm', 'x': '24.655mm', 'y': '95.614mm'}
{'layout': 'tb', 'y': '3.574mm'}
{'name': 'SubAwardee'}
{'minH': '54.07mm', 'name': 'LobbyingRegistrant', 'w': '171.408mm', 'x': '24.655mm', 'y': '161.716mm'}
{'name': 'Address', 'x': '0.17mm', 'y': '12.07mm'}
{'name': 'Individual', 'w': '171.4082mm', 'y': '31.228mm'}
{'name': 'Address', 

In [19]:
# remove signature form field
for field in root.findall(".//xfa:subform[@name='SFLLL_2_0_P1']/xfa:field", ns):
    print(field.attrib)
    if field.get('name') == 'Signature':
        field.getparent().remove(field)

print("\n")

for field in root.findall(".//xfa:subform[@name='SFLLL_2_0_P1']/xfa:field", ns):
    print(field.attrib)

{'h': '0.0035mm', 'name': 'XDPFirstField', 'w': '0.0035mm'}
{'h': '6mm', 'name': 'Mandatory', 'presence': 'hidden', 'w': '20.9126mm', 'x': '39.119mm', 'y': '4.799mm'}
{'h': '4.5002mm', 'name': 'CloseForm', 'relevant': '-print', 'w': '62.7101mm', 'x': '82.276mm', 'y': '24.206mm'}
{'h': '4.572mm', 'name': 'FedDepAgency', 'w': '84.9376mm', 'x': '24.585mm', 'y': '129.806mm'}
{'h': '6.78mm', 'name': 'FedProgName', 'w': '85.4498mm', 'x': '113.015mm', 'y': '129.806mm'}
{'h': '4.572mm', 'name': 'CfdaNumber', 'w': '52.12mm', 'x': '146.345mm', 'y': '137.376mm'}
{'h': '6.78mm', 'name': 'FedActionNumber', 'w': '84.9298mm', 'x': '24.625mm', 'y': '147.717mm'}
{'h': '4.572mm', 'name': 'AwardAmt', 'w': '37.555mm', 'x': '117.945mm', 'y': '147.966mm'}
{'h': '4.572mm', 'name': 'Prefix3', 'w': '19.05mm', 'x': '48.165mm', 'y': '236.376mm'}
{'h': '4.572mm', 'name': 'First3', 'w': '46.02mm', 'x': '82.565mm', 'y': '236.376mm'}
{'h': '4.572mm', 'name': 'Middle3', 'w': '40.21mm', 'x': '144.315mm', 'y': '236.376

In [20]:
# create the signature field
# copy attribs from Signature field in old form
field = etree.Element("{http://www.xfa.org/schema/xfa-template/3.3/}field", attrib = {'minH': '6.35mm', 'name': 'Signature', 'w': '88.1068mm', 'x': '40.5102mm', 'y': '229.1299mm'}
)
ui = etree.SubElement(field, "{http://www.xfa.org/schema/xfa-template/3.3/}ui")
sig = etree.SubElement(ui, "{http://www.xfa.org/schema/xfa-template/3.3/}signature", attrib = {'type':'PDF1.6'})
fil = etree.SubElement(sig, "{http://www.xfa.org/schema/xfa-template/3.3/}filter")
loc = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}lockDocument")#, attrib = {'type': 'required'})
loc.text = "1"
#mdp = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}mdp", attrib = {'permissions': '1', 'signatureType': 'author'})
print(etree.tostring(field))

b'<ns0:field xmlns:ns0="http://www.xfa.org/schema/xfa-template/3.3/" minH="6.35mm" name="Signature" w="88.1068mm" x="40.5102mm" y="229.1299mm"><ns0:ui><ns0:signature type="PDF1.6"><ns0:filter><ns0:lockDocument>1</ns0:lockDocument></ns0:filter></ns0:signature></ns0:ui></ns0:field>'


In [21]:
# now add the signature field
root.find(".//xfa:subform[@name='SFLLL_2_0_P1']", ns).append(field)

for field in root.findall(".//xfa:subform[@name='SFLLL_2_0_P1']/xfa:field", ns):
    print(field.attrib)

{'h': '0.0035mm', 'name': 'XDPFirstField', 'w': '0.0035mm'}
{'h': '6mm', 'name': 'Mandatory', 'presence': 'hidden', 'w': '20.9126mm', 'x': '39.119mm', 'y': '4.799mm'}
{'h': '4.5002mm', 'name': 'CloseForm', 'relevant': '-print', 'w': '62.7101mm', 'x': '82.276mm', 'y': '24.206mm'}
{'h': '4.572mm', 'name': 'FedDepAgency', 'w': '84.9376mm', 'x': '24.585mm', 'y': '129.806mm'}
{'h': '6.78mm', 'name': 'FedProgName', 'w': '85.4498mm', 'x': '113.015mm', 'y': '129.806mm'}
{'h': '4.572mm', 'name': 'CfdaNumber', 'w': '52.12mm', 'x': '146.345mm', 'y': '137.376mm'}
{'h': '6.78mm', 'name': 'FedActionNumber', 'w': '84.9298mm', 'x': '24.625mm', 'y': '147.717mm'}
{'h': '4.572mm', 'name': 'AwardAmt', 'w': '37.555mm', 'x': '117.945mm', 'y': '147.966mm'}
{'h': '4.572mm', 'name': 'Prefix3', 'w': '19.05mm', 'x': '48.165mm', 'y': '236.376mm'}
{'h': '4.572mm', 'name': 'First3', 'w': '46.02mm', 'x': '82.565mm', 'y': '236.376mm'}
{'h': '4.572mm', 'name': 'Middle3', 'w': '40.21mm', 'x': '144.315mm', 'y': '236.376

In [22]:
# update pdf data
xfaDict['template'] = etree.tostring(root)
pdfData.save("SFLLL_signable.pdf")

In [30]:
pdfData.close()

In [31]:
pdfData = pikepdf.Pdf.open("SF424_4_0-V4.0 (4).pdf")
xfaDict = XfaObj(pdfData)
root = etree.fromstring(xfaDict['template'])
ns = {'xfa':"http://www.xfa.org/schema/xfa-template/3.3/"}

In [32]:
for field in root.findall(".//xfa:subform[@name='SF424_4_0_P3']/xfa:field", ns):
    print(field.attrib)

{'h': '0.6728mm', 'name': 'Mandatory', 'presence': 'hidden', 'w': '20.9126mm', 'x': '43.1299mm', 'y': '0.7956mm'}
{'h': '4.7978mm', 'name': 'CongressionalDistrictApplicant', 'w': '18.79mm', 'x': '34.58mm', 'y': '38.63mm'}
{'h': '4.7978mm', 'name': 'CongressionalDistrictProgramProject', 'w': '18.79mm', 'x': '145.2601mm', 'y': '38.63mm'}
{'h': '5.0102mm', 'name': 'Single_Attach0', 'w': '71.7mm', 'x': '12.44mm', 'y': '51.09mm'}
{'h': '5.5383mm', 'name': 'AddSingleAttach0', 'w': '27.7804mm', 'x': '86.87mm', 'y': '50.56mm'}
{'h': '5.5383mm', 'name': 'DeleteSingleAttach0', 'w': '27.7804mm', 'x': '116.8828mm', 'y': '50.56mm'}
{'h': '5.5383mm', 'name': 'ViewSingleAttach0', 'w': '27.7804mm', 'x': '146.6499mm', 'y': '50.56mm'}
{'h': '4.7978mm', 'name': 'ProjectStartDate', 'w': '18.7901mm', 'x': '32.34mm', 'y': '64.32mm'}
{'h': '4.7978mm', 'name': 'ProjectEndDate', 'w': '18.7901mm', 'x': '145.8mm', 'y': '64.32mm'}
{'h': '4.7978mm', 'name': 'FederalEstimatedFunding', 'w': '48.9384mm', 'x': '37.703

In [33]:
# remove signature form field
for field in root.findall(".//xfa:subform[@name='SF424_4_0_P3']/xfa:field", ns):
    print(field.attrib)
    if field.get('name') == 'AORSignature':
        field.getparent().remove(field)

print("\n")

for field in root.findall(".//xfa:subform[@name='SF424_4_0_P3']/xfa:field", ns):
    print(field.attrib)

{'h': '0.6728mm', 'name': 'Mandatory', 'presence': 'hidden', 'w': '20.9126mm', 'x': '43.1299mm', 'y': '0.7956mm'}
{'h': '4.7978mm', 'name': 'CongressionalDistrictApplicant', 'w': '18.79mm', 'x': '34.58mm', 'y': '38.63mm'}
{'h': '4.7978mm', 'name': 'CongressionalDistrictProgramProject', 'w': '18.79mm', 'x': '145.2601mm', 'y': '38.63mm'}
{'h': '5.0102mm', 'name': 'Single_Attach0', 'w': '71.7mm', 'x': '12.44mm', 'y': '51.09mm'}
{'h': '5.5383mm', 'name': 'AddSingleAttach0', 'w': '27.7804mm', 'x': '86.87mm', 'y': '50.56mm'}
{'h': '5.5383mm', 'name': 'DeleteSingleAttach0', 'w': '27.7804mm', 'x': '116.8828mm', 'y': '50.56mm'}
{'h': '5.5383mm', 'name': 'ViewSingleAttach0', 'w': '27.7804mm', 'x': '146.6499mm', 'y': '50.56mm'}
{'h': '4.7978mm', 'name': 'ProjectStartDate', 'w': '18.7901mm', 'x': '32.34mm', 'y': '64.32mm'}
{'h': '4.7978mm', 'name': 'ProjectEndDate', 'w': '18.7901mm', 'x': '145.8mm', 'y': '64.32mm'}
{'h': '4.7978mm', 'name': 'FederalEstimatedFunding', 'w': '48.9384mm', 'x': '37.703

In [34]:
# create the signature field
# copy attribs from Signature field in old form
field = etree.Element("{http://www.xfa.org/schema/xfa-template/3.3/}field", attrib = {'h': '11.621mm', 'name': 'AORSignature', 'w': '90.483mm', 'x': '64.56mm', 'y': '254.55mm'})
ui = etree.SubElement(field, "{http://www.xfa.org/schema/xfa-template/3.3/}ui")
sig = etree.SubElement(ui, "{http://www.xfa.org/schema/xfa-template/3.3/}signature", attrib = {'type':'PDF1.6'})
fil = etree.SubElement(sig, "{http://www.xfa.org/schema/xfa-template/3.3/}filter")
loc = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}lockDocument")#, attrib = {'type': 'required'})
loc.text = "1"
#mdp = etree.SubElement(fil, "{http://www.xfa.org/schema/xfa-template/3.3/}mdp", attrib = {'permissions': '1', 'signatureType': 'author'})
print(etree.tostring(field))

b'<ns0:field xmlns:ns0="http://www.xfa.org/schema/xfa-template/3.3/" h="11.621mm" name="AORSignature" w="90.483mm" x="64.56mm" y="254.55mm"><ns0:ui><ns0:signature type="PDF1.6"><ns0:filter><ns0:lockDocument>1</ns0:lockDocument></ns0:filter></ns0:signature></ns0:ui></ns0:field>'


In [35]:
# now add the signature field
root.find(".//xfa:subform[@name='SF424_4_0_P3']", ns).append(field)

for field in root.findall(".//xfa:subform[@name='SF424_4_0_P3']/xfa:field", ns):
    print(field.attrib)

{'h': '0.6728mm', 'name': 'Mandatory', 'presence': 'hidden', 'w': '20.9126mm', 'x': '43.1299mm', 'y': '0.7956mm'}
{'h': '4.7978mm', 'name': 'CongressionalDistrictApplicant', 'w': '18.79mm', 'x': '34.58mm', 'y': '38.63mm'}
{'h': '4.7978mm', 'name': 'CongressionalDistrictProgramProject', 'w': '18.79mm', 'x': '145.2601mm', 'y': '38.63mm'}
{'h': '5.0102mm', 'name': 'Single_Attach0', 'w': '71.7mm', 'x': '12.44mm', 'y': '51.09mm'}
{'h': '5.5383mm', 'name': 'AddSingleAttach0', 'w': '27.7804mm', 'x': '86.87mm', 'y': '50.56mm'}
{'h': '5.5383mm', 'name': 'DeleteSingleAttach0', 'w': '27.7804mm', 'x': '116.8828mm', 'y': '50.56mm'}
{'h': '5.5383mm', 'name': 'ViewSingleAttach0', 'w': '27.7804mm', 'x': '146.6499mm', 'y': '50.56mm'}
{'h': '4.7978mm', 'name': 'ProjectStartDate', 'w': '18.7901mm', 'x': '32.34mm', 'y': '64.32mm'}
{'h': '4.7978mm', 'name': 'ProjectEndDate', 'w': '18.7901mm', 'x': '145.8mm', 'y': '64.32mm'}
{'h': '4.7978mm', 'name': 'FederalEstimatedFunding', 'w': '48.9384mm', 'x': '37.703

In [36]:
# update pdf data
xfaDict['template'] = etree.tostring(root)
pdfData.save("SF424_General_signable.pdf")

In [36]:
pdfData.close()

In [38]:
!python "C:\Users\ian.berg\Documents\GitHub\pdf-xfa-tools\xfa-extract.py" "C:\Temp\ss4a"

In [14]:
xfaDict.keys()

dict_keys(['xdp:xdp', 'config', 'template', 'connectionSet', 'datasets', 'xmpmeta', 'xfdf', 'form', '</xdp:xdp>'])

In [19]:
xfaDict["config"]

'<config xmlns="http://www.xfa.org/schema/xci/3.0/"\n><agent name="designer"\n><!--  [0..n]  --><destination\n>pdf</destination\n><pdf\n><!--  [0..n]  --><fontInfo\n/></pdf\n></agent\n><present\n><!--  [0..n]  --><pdf\n><!--  [0..n]  --><version\n>1.7</version\n><adobeExtensionLevel\n>8</adobeExtensionLevel\n><renderPolicy\n>client</renderPolicy\n><creator\n>Designer 6.4</creator\n><producer\n>Designer 6.4</producer\n><scriptModel\n>XFA</scriptModel\n><interactive\n>1</interactive\n><tagged\n>1</tagged\n><fontInfo\n><embed\n>0</embed\n></fontInfo\n><compression\n><level\n>6</level\n><compressLogicalStructure\n>1</compressLogicalStructure\n><compressObjectStream\n>1</compressObjectStream\n></compression\n><linearized\n>1</linearized\n><silentPrint\n><addSilentPrint\n>0</addSilentPrint\n><printerName\n/></silentPrint\n><viewerPreferences\n><duplexOption\n>simplex</duplexOption\n><numberOfCopies\n>0</numberOfCopies\n><printScaling\n>appDefault</printScaling\n><pickTrayByPDFSize\n>0</pickT

In [35]:
pdfData.Root.AcroForm.Fields

<pikepdf.Array([ {
  "/Kids": [ {
    "/Kids": [ {
      "/Kids": [ {
        "/AP": {
          "/N": pikepdf.Stream(owner=<...>, data=<...>, {
            "/BBox": [ Decimal('0.0'), Decimal('0.0'), Decimal('256.487'), Decimal('32.941') ],
            "/Length": 7,
            "/Resources": {
              "/XObject": {
                "/FRM": pikepdf.Stream(owner=<...>, data=<...>, {
                  "/BBox": [ Decimal('0.0'), Decimal('0.0'), Decimal('256.487'), Decimal('32.941') ],
                  "/Length": 52,
                  "/Resources": {
                    "/XObject": {
                      "/n0": pikepdf.Stream(owner=<...>, data=<...>, {
                        "/BBox": [ Decimal('0.0'), Decimal('0.0'), Decimal('100.0'), Decimal('100.0') ],
                        "/Length": 10,
                        "/Resources": {

                        },
                        "/Subtype": "/Form",
                        "/Type": "/XObject"
                      }),
          

In [39]:
pdfData = pikepdf.Pdf.open(r"C:\Temp\SS4A\SF424_SS4A (1).pdf")
xfaDict = XfaObj(pdfData)

In [62]:
pdfData.Root.Pages.Kids[2].Annots

<pikepdf.Array([ {
  "/AP": {
    "/N": pikepdf.Stream(owner=<...>, data=<...>, {
      "/BBox": [ Decimal('0.0'), Decimal('0.0'), Decimal('256.487'), Decimal('32.941') ],
      "/Length": 7,
      "/Resources": {
        "/XObject": {
          "/FRM": pikepdf.Stream(owner=<...>, data=<...>, {
            "/BBox": [ Decimal('0.0'), Decimal('0.0'), Decimal('256.487'), Decimal('32.941') ],
            "/Length": 52,
            "/Resources": {
              "/XObject": {
                "/n0": pikepdf.Stream(owner=<...>, data=<...>, {
                  "/BBox": [ Decimal('0.0'), Decimal('0.0'), Decimal('100.0'), Decimal('100.0') ],
                  "/Length": 10,
                  "/Resources": {

                  },
                  "/Subtype": "/Form",
                  "/Type": "/XObject"
                }),
                "/n2": pikepdf.Stream(owner=<...>, data=<...>, {
                  "/BBox": [ Decimal('1.0'), Decimal('1.0'), Decimal('255.487'), <...> ],
                  "/

In [4]:
root = etree.fromstring(xfaDict["datasets"])
    
ns = {'h':"http://apply.grants.gov/system/Header-V1.0", 
      'sf424': "http://apply.grants.gov/forms/SF424_4_0-V4.0", 
      'f': "http://apply.grants.gov/system/Footer-V1.0",
      'xfa': "http://www.xfa.org/schema/xfa-data/1.0/",
      'meta': 'http://apply.grants.gov/system/MetaGrantApplicationWrapper',
      'meta2': 'http://apply.grants.gov/system/MetaGrantApplication'}    

sf424 = root.find("xfa:data",ns).find("meta:GrantApplicationWrapper",ns).find(
    "meta2:GrantApplication",ns).find('{http://apply.grants.gov/system/MetaGrantApplication}Forms').find(
    'sf424:SF424_4_0', ns)

sf424

<Element {http://apply.grants.gov/forms/SF424_4_0-V4.0}SF424_4_0 at 0x19de0a8df80>

In [5]:
#for child in sf424:
#    child.text = "AAAAA"
for child in sf424:
    print(child.tag, child.text)

{http://apply.grants.gov/forms/SF424_4_0-V4.0}SubmissionType Application
{http://apply.grants.gov/forms/SF424_4_0-V4.0}ApplicationType New
{http://apply.grants.gov/forms/SF424_4_0-V4.0}DateReceived 2024-04-04
{http://apply.grants.gov/forms/SF424_4_0-V4.0}OrganizationName Village of Stevensville
{http://apply.grants.gov/forms/SF424_4_0-V4.0}EmployerTaxpayerIdentificationNumber 38-1684880
{http://apply.grants.gov/forms/SF424_4_0-V4.0}SAMUEI FEEYLNUB29J6
{http://apply.grants.gov/forms/SF424_4_0-V4.0}Applicant None
{http://apply.grants.gov/forms/SF424_4_0-V4.0}ContactPerson None
{http://apply.grants.gov/forms/SF424_4_0-V4.0}PhoneNumber 269-429-1802
{http://apply.grants.gov/forms/SF424_4_0-V4.0}Email manager@villageofstevensville.us
{http://apply.grants.gov/forms/SF424_4_0-V4.0}ApplicantTypeCode1 C: City or Township Government
{http://apply.grants.gov/forms/SF424_4_0-V4.0}AgencyName Department of Transportation (DOT)
{http://apply.grants.gov/forms/SF424_4_0-V4.0}FundingOpportunityNumber DOT

In [7]:
#for child in sf424:
#    child.text = "AAAAA"
for child in sf424:
    print(child.tag, child.text)

{http://apply.grants.gov/forms/SF424_4_0-V4.0}SubmissionType Application
{http://apply.grants.gov/forms/SF424_4_0-V4.0}ApplicationType New
{http://apply.grants.gov/forms/SF424_4_0-V4.0}DateReceived 2024-04-04
{http://apply.grants.gov/forms/SF424_4_0-V4.0}OrganizationName Village of Stevensville
{http://apply.grants.gov/forms/SF424_4_0-V4.0}EmployerTaxpayerIdentificationNumber 38-1684880
{http://apply.grants.gov/forms/SF424_4_0-V4.0}SAMUEI FEEYLNUB29J6
{http://apply.grants.gov/forms/SF424_4_0-V4.0}Applicant None
{http://apply.grants.gov/forms/SF424_4_0-V4.0}ContactPerson None
{http://apply.grants.gov/forms/SF424_4_0-V4.0}PhoneNumber 269-429-1802
{http://apply.grants.gov/forms/SF424_4_0-V4.0}Email manager@villageofstevensville.us
{http://apply.grants.gov/forms/SF424_4_0-V4.0}ApplicantTypeCode1 C: City or Township Government
{http://apply.grants.gov/forms/SF424_4_0-V4.0}AgencyName Department of Transportation (DOT)
{http://apply.grants.gov/forms/SF424_4_0-V4.0}FundingOpportunityNumber DOT

In [16]:
#ET.tostring(sf424)
sf424.find("{http://apply.grants.gov/forms/SF424_4_0-V4.0}ApplicantTypeCode1").text = 'X: Other (specify)' 

In [55]:
xfaDict['datasets'] = ET.tostring(root)

In [56]:
pdfData.save("test_424.pdf")

In [22]:
os.getcwd()

'C:\\Users\\ian.berg\\Documents\\GitHub\\pdf-xfa-tools'

In [30]:

#xfaDict['template']
dataSoup = BeautifulSoup(xfaDict['datasets'],'xml')
tag = dataSoup.find("AuthorizedRepresentative")
tag

<SF424_4_0:AuthorizedRepresentative><globLib:FirstName xmlns:globLib="http://apply.grants.gov/system/GlobalLibrary-V2.0"/><globLib:LastName xmlns:globLib="http://apply.grants.gov/system/GlobalLibrary-V2.0"/></SF424_4_0:AuthorizedRepresentative>

In [27]:
tag.contents

[]

In [28]:
tag

<globLib:FirstName xmlns:globLib="http://apply.grants.gov/system/GlobalLibrary-V2.0"/>

In [22]:
dir(tag)

['DEFAULT_INTERESTING_STRING_TYPES',
 'EMPTY_ELEMENT_EVENT',
 'END_ELEMENT_EVENT',
 'START_ELEMENT_EVENT',
 'STRING_ELEMENT_EVENT',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_clone',
 '_event_stream',
 '_find_all',
 '_find_one',
 '_format_tag',
 '_indent_string',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'chi

In [23]:
pdfData.close()

In [4]:
import pyhanko

In [6]:
from pyhanko.pdf_utils.reader import PdfFileReader

doc= open(r"C:\Users\ian.berg\Documents\GitHub\pdf-xfa-tools\SF424_General_signable_ib.pdf", 'rb') 
r = PdfFileReader(doc)
print([{sig.field_name: sig.signer_cert.subject.human_friendly} for sig in r.embedded_signatures])
doc.close()



[{'GrantApplicationWrapper[0].SF424_4_0_Main[0].SF424_4_0_P3[0].AORSignature[0]': 'Common Name: IAN ANDREW BERG; Organizational Unit: 540168, Department of Transportation; Organization: U.S. Government; Country: US'}]


In [5]:
sig = r.embedded_signatures[0]
sig.field_name

'GrantApplicationWrapper[0].SF424_4_0_Main[0].SF424_4_0_P3[0].AORSignature[0]'

TypeError: encoded_data must be a byte string, not str

[('version', asn1crypto.cms.CMSVersion, {}),
 ('sid', asn1crypto.cms.SignerIdentifier, {}),
 ('digest_algorithm', asn1crypto.algos.DigestAlgorithm, {}),
 ('signed_attrs',
  asn1crypto.cms.CMSAttributes,
  {'implicit': 0, 'optional': True}),
 ('signature_algorithm', asn1crypto.algos.SignedDigestAlgorithm, {}),
 ('signature', asn1crypto.core.OctetString, {}),
 ('unsigned_attrs',
  asn1crypto.cms.CMSAttributes,
  {'implicit': 1, 'optional': True})]